In [1]:
!pip install torch # 앞에 '!'가 붙는 명령은 파이썬이 아니라 Shell script로 실행됨. 파이토치 설치 명령

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch

### 파이토치에서 GPU를 왜, 어떻게 쓰나요?

**CPU,GPU 관련 상대 속도 개요:**

```
┌───────────┐            ┌───────────┐
│    CPU    │ ─────────► │    GPU    │
│   느리지만  │ 느린 전송    │ 빠르지만    │
│다양한 연산  │ ◄────────  │ 특수 연산만  │
└────┬─▲────┘            └────┬─▲────┘
     │ │ 빠른 전송              │ │ 아주 빠른 전송
  ┌──▼─┴──┐                ┌──▼─┴───┐
  │  RAM  │                │  VRAM  │
  │       │                │        │
  └───────┘                └────────┘
```

## 어떻게?

파이토치는 기본적으로 새로운 텐서 (추가적인 기능이 있는 숫자 데이터 형식) 를 시스템 RAM에 생성하고, CPU 로 연산을 수행합니다.

In [3]:
a = torch.tensor([2.,3.])

In [4]:
a.device 

device(type='cpu')

보시다시피 'a'라는 텐서는 'cpu'라는 'device'에 저장되어있습니다.

지금 GPU 를 활용할 수 있는지 확인해볼까요?

In [5]:
torch.cuda.is_available()

True

가능하다면, 현재 'cpu'에 거주하는 'a' 텐서를 'cuda'라고 불리는 GPU로 옮겨봅시다

In [6]:
if torch.cuda.is_available():
    a = a.cuda()
    a = a.to('cuda:0') # 위와 동일한 기능
    print(a.device)

cuda:0


이제 텐서 저장 위치가 'cuda:0'로 바뀌었습니다. 끝에 '0'은 "첫번째 GPU' 를 뜻합니다 (한대의 컴퓨터에 수개의 GPU 장착이 가능하기 때문에)

## 왜?

그럼, CPU와 GPU 속도의 차이를 실감나게 테스트해보죠. 간단하게 10000x10000 랜덤 2차원 배열을 세제곱 합니다.

In [7]:
torch.randn((2,2))

tensor([[-1.4877,  0.2730],
        [ 0.7363,  0.0047]])

In [8]:
%%timeit
b = torch.randn((10000,10000), device='cpu')
b ** 3

559 ms ± 1.63 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
b = torch.randn((10000,10000), device='cuda:0')
b ** 3

3.39 ms ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 거의 ***200배*** 빠르네요!

# 기타 정보

설치된 GPU 상태를 보려면, 다음 명령을 사용해요

In [10]:
!nvidia-smi

Sat Dec  4 22:38:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   52C    P8    21W / 235W |   3958MiB /  7948MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------